In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=68600 sha256=395b527d3538802e4a7cc8095e742a261cf06b9513645b59f13e10d820f4d0e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-gldqkacg/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.invariant_train import ERM

model = model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:03<00:00, 203.95batch/s, accuracy=100.0%, loss=0.0119]  


In [5]:
from spuco.group_inference import CorrectNContrastInference

model = model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
cnc_inference = CorrectNContrastInference(
    trainset=trainset, 
    model=model, 
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=1,
    device=device,
    verbose=True
)

In [6]:
group_partition = cnc_inference.infer_groups()
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

Getting Trainset Outputs: 100%|██████████| 751/751 [00:00<00:00, 1258.32batch/s]

(0, 0) 10124
(0, 1) 9673
(0, 2) 9021
(0, 3) 9746
(0, 4) 9440


In [7]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.08139075464243
Group (0, 1) Test Accuracy: 98.98687067093972
Group (0, 2) Test Accuracy: 98.88039020064295
Group (0, 3) Test Accuracy: 99.00471988508106
Group (0, 4) Test Accuracy: 99.0042372881356


{(0, 0): 99.08139075464243,
 (0, 1): 98.98687067093972,
 (0, 2): 98.88039020064295,
 (0, 3): 99.00471988508106,
 (0, 4): 99.0042372881356}

In [28]:
from torch.optim import SGD
from spuco.invariant_train import CorrectNContrastTrain
from spuco.models import model_factory 
from spuco.datasets import GroupLabeledDatasetWrapper

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
cnc_train = CorrectNContrastTrain(
    trainset=GroupLabeledDatasetWrapper(trainset, group_partition),
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, weight_decay=5e-4),
    num_pos=100,
    num_neg=100,
    num_epochs=5,
    lambda_ce=0.01,
    device=device,
    verbose=True
)
cnc_train.train()

Epoch 4: 100%|██████████| 751/751 [00:33<00:00, 22.56batch/s, accuracy=100.0%, loss=0.00744]  


In [29]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 0.0
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 0.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 100.0
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 0.0
Group (3, 3) Test Accuracy: 100.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [30]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [31]:
evaluator.average_accuracy

98.99383384717939

In [32]:
evaluator.evaluate_spurious_task()

100.0